# Лабораторная работа №1

выполнила: Рыжкова Ульяна БВТ2001

In [13]:
import numpy as np
import torch
import torch.nn.functional as F

def myconv2d(inputs, weight, bias=None, stride=1 , padding=0,  groups=1, dilation=1):
    # Get the dimensions of the input and kernel
    batch_size, in_channels, in_height, in_width = inputs.shape
    out_channels, _, kernel_height, kernel_width = weight.shape

    # Apply dilation to the kernel
    dilated_kernel = np.zeros((out_channels, in_channels, (kernel_height - 1) * dilation + 1, (kernel_width - 1) * dilation + 1))
    dilated_kernel[:, :, ::dilation, ::dilation] = weight

    #Add padding
    if padding > 0:
        inputs = F.pad(inputs, (padding, padding, padding, padding))

    # Compute the output dimensions
    out_height = (in_height + 2 * padding - dilated_kernel.shape[2]) // stride + 1
    out_width = (in_width + 2 * padding - dilated_kernel.shape[3]) // stride + 1

    # Create the output tensor
    output = torch.zeros((batch_size, out_channels, out_height, out_width))

    #2D convolution for each item in the batch
    for b in range(batch_size):
        for oc in range(out_channels):
            for oh in range(out_height):
                for ow in range(out_width):
                    if bias is not None:
                        output[b, oc, oh, ow] = (inputs[b, :, oh*stride:oh*stride+dilated_kernel.shape[2], ow*stride:ow*stride+dilated_kernel.shape[3]] * dilated_kernel[oc]).sum() + bias[oc]
                    else:
                        output[b, oc, oh, ow] = (inputs[b, :, oh*stride:oh*stride+dilated_kernel.shape[2], ow*stride:ow*stride+dilated_kernel.shape[3]] * dilated_kernel[oc]).sum()

    return output


# Generate inputs
batch_size, in_channels, height, width = 1, 1, 4, 4
out_channels = 1
kernel_size = 3

input_data = torch.randn(batch_size, in_channels, height, width)
weight_data = torch.randn(out_channels, in_channels, kernel_size, kernel_size)
bias_data = torch.randn(out_channels)

# Use function
output = myconv2d(input_data, weight_data, bias_data, stride=1, padding=1, dilation=1, groups=1)


In [14]:
import ipytest
ipytest.autoconfig()

In [21]:
%%ipytest


def test_1():
    batch_size, in_channels, height, width = 2, 3, 4, 4
    out_channels = 2
    kernel_size = 3

    input_data = torch.randn(batch_size, in_channels, height, width)
    weight_data = torch.randn(out_channels, in_channels, kernel_size, kernel_size)        


    conv = F.conv2d(input_data, weight_data, bias=None, stride=1, padding=0)
    my_conv = myconv2d(input_data, weight_data, bias=None, stride=1, padding=0)
    assert torch.allclose(conv, my_conv)

def test_2():
    batch_size, in_channels, height, width = 2, 3, 4, 4
    out_channels = 2
    kernel_size = 3

    input_data = torch.randn(batch_size, in_channels, height, width)
    weight_data = torch.randn(out_channels, in_channels, kernel_size, kernel_size)        
    bias_data = torch.randn(out_channels)

    conv = F.conv2d(input_data, weight_data, bias_data, stride=1, padding=3)
    my_conv = myconv2d(input_data, weight_data, bias_data, stride=1, padding=3)
    assert torch.allclose(conv, my_conv)


def test_3():
    batch_size, in_channels, height, width = 2, 3, 4, 4
    out_channels = 2
    kernel_size = 3

    input_data = torch.randn(batch_size, in_channels, height, width)
    weight_data = torch.randn(out_channels, in_channels, kernel_size, kernel_size)        
    bias_data = torch.randn(out_channels)

    conv = F.conv2d(input_data, weight_data, bias_data, stride=2, padding=0)
    my_conv = myconv2d(input_data, weight_data, bias_data, stride=2, padding=0)
    assert torch.allclose(conv, my_conv)

def test_4():
    batch_size, in_channels, height, width = 2, 3, 4, 4
    out_channels = 2
    kernel_size = 3
    dilation = 2

    input_data = torch.randn(batch_size, in_channels, height+dilation, width+dilation)
    weight_data = torch.randn(out_channels, in_channels, kernel_size, kernel_size)        
    bias_data = torch.randn(out_channels)

    conv = F.conv2d(input_data, weight_data, bias_data, stride=1, padding=0, dilation=dilation)
    my_conv = myconv2d(input_data, weight_data, bias_data, stride=1, padding=0, dilation=dilation)
    assert torch.allclose(conv, my_conv)


....                                                                                         [100%]
4 passed in 0.13s
